In [1]:

from env.res_env_all import ResistanceFinalEnv
from env.naive_agents import NaiveEvilPolicy, RandomPolicy

from runner.runner_all import RunnerMA_Both

import numpy as np
import os
import numpy as np

from runner.game import simulate_game
from runner.config import Args

### Restore trained policies

In [2]:


env = ResistanceFinalEnv()
config = {
    'all_args': Args(),
    'env': env,
    'device': 'cpu',
}
runner = RunnerMA_Both(config)

runner.restore('models', version='temp')



evil_policy = runner.trainer_evil.policy

good_policy = runner.trainer_good.policy


### Simulate a Game

In [3]:
config = {
            0: good_policy,
            1: NaiveEvilPolicy()
        }
env = ResistanceFinalEnv()
simulate_game(env, config, verbose=True)

-- Start of the Game --
Good Team: ['player_1', 'player_4', 'player_5']
Evil Team: ['player_2', 'player_3']
Round: 0, Phase: 0, Vote attempt 1
player_4 😇: [0, 1, 0, 0, 1] (1.00)
Rewards
player_4 😇: -0.0
Round: 0, Phase: 1, Vote attempt 1
player_1 😇: 0 (1.00)
player_2 😈: 1 (None)
player_3 😈: 1 (None)
player_4 😇: 0 (1.00)
player_5 😇: 1 (1.00)
Rewards
player_1 😇: 0.0
player_2 😈: 0
player_3 😈: 0
player_4 😇: 0.0
player_5 😇: -0.0
Round: 0, Phase: 2, Vote attempt 1
player_2 😈: 1 (None)
player_5 😇: 0 (1.00)
 ---> Round 0: 😇 0 : 1 😈
Rewards
player_2 😈: 0
player_5 😇: 0
Round: 1, Phase: 0, Vote attempt 1
player_5 😇: [1, 0, 0, 1, 1] (0.99)
Rewards
player_5 😇: 0.15
Round: 1, Phase: 1, Vote attempt 1
player_1 😇: 0 (1.00)
player_2 😈: 0 (None)
player_3 😈: 0 (None)
player_4 😇: 0 (1.00)
player_5 😇: 0 (0.94)
Rewards
player_1 😇: -0.15000000000000002
player_2 😈: 0
player_3 😈: 0
player_4 😇: -0.15000000000000002
player_5 😇: -0.15000000000000002
Round: 1, Phase: 0, Vote attempt 2
player_1 😇: [1, 1, 0, 1, 0] (

{'number of turns': 21, 'good victory': 0, 'number of rounds': 3}

### Simulate n games

In [4]:
n_games = 1000
good_victories = 0
number_of_turns = 0
number_of_success = 0
for i in range(n_games):
    stat = simulate_game(env, config)
    good_victories += stat['good victory']
    number_of_turns  += stat['number of turns']
    if stat['good victory'] == 1:
        number_of_success += 3
    else:
        number_of_success += (stat['number of rounds'] - 3)
    
print('Good Victories Rate', good_victories / n_games)
print('Av number of turns', number_of_turns/ n_games)
print('Av number of succ Quests', number_of_success/ n_games)

Good Victories Rate 0.182
Av number of turns 26.64
Av number of succ Quests 0.821


config = {
            0: good_policy,
            1: NaiveEvilPolicy()
        }
simulate_game(env, config, verbose=True)

n_games = 100
good_victories = 0
number_of_turns = 0
for i in range(n_games):
    stat = simulate_game(env, config)
    good_victories += stat['good victory']
    number_of_turns  += stat['number of turns']
print('Good Victories Rate', good_victories / n_games)
print('Av number of turns', number_of_turns/ n_games)

config = {
            0: good_policy,
            1: RandomPolicy()
        }
env = ResistanceFinalEnv()
n_games = 1000
good_victories = 0
number_of_turns = 0
number_of_success = 0
for i in range(n_games):
    stat = simulate_game(env, config)
    good_victories += stat['good victory']
    number_of_turns  += stat['number of turns']
    if stat['good victory'] == 1:
        number_of_success += 3
    else:
        number_of_success += (stat['number of rounds'] - 3)

print('Good vs Random: ')
print('Good Victories Rate', good_victories / n_games)
print('Av number of turns', number_of_turns/ n_games)
print('Av number of succ Quests', number_of_success/ n_games)

config = {
            0: RandomPolicy(),
            1: evil_policy
        }
env = ResistanceFinalEnv()
n_games = 1000
good_victories = 0
number_of_turns = 0
number_of_success = 0
for i in range(n_games):
    stat = simulate_game(env, config)
    good_victories += stat['good victory']
    number_of_turns  += stat['number of turns']
    if stat['good victory'] == 1:
        number_of_success += 3
    else:
        number_of_success += (stat['number of rounds'] - 3)

print('Evil vs Random: ')
print('Good Victories Rate', good_victories / n_games)
print('Av number of turns', number_of_turns/ n_games)
print('Av number of succ Quests', number_of_success/ n_games)

In [5]:
env.evil_players

['player_3', 'player_5']

In [6]:
env.quest_team = [0, 1, 0, 0, 0]

In [7]:
env.reset()
env.evil_players

['player_1', 'player_3']

In [8]:
def _is_evil_in_quest_team():
    for evil_player in env.evil_players:
        evil_player_idx = int(evil_player[-1]) - 1
        if env.quest_team[evil_player_idx] == 1:
            return True
    return False

In [9]:
_is_evil_in_quest_team() == False

True

In [10]:
import torch.nn as nn
def init(module, weight_init, bias_init, gain=1):
    weight_init(module.weight.data, gain=gain)
    if module.bias is not None:
        bias_init(module.bias.data)
    return module

class MLPLayer(nn.Module):
    def __init__(self, input_dim, layer_sizes: list, use_orthogonal, use_ReLU):
        super(MLPLayer, self).__init__()
        active_func = [nn.Tanh(), nn.ReLU()][use_ReLU]
        init_method = [nn.init.xavier_uniform_, nn.init.orthogonal_][use_orthogonal]
        gain = nn.init.calculate_gain(['tanh', 'relu'][use_ReLU])

        def init_(m):
            return init(m, init_method, lambda x: nn.init.constant_(x, 0), gain=gain)

        self.layers = nn.ModuleList()
        prev_size = input_dim
        for size in layer_sizes:
            fc = nn.Linear(prev_size, size)
            init_(fc)
            self.layers.append(
                nn.Sequential(fc, active_func, nn.LayerNorm(size))
            )
            prev_size = size

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


class MLPBase(nn.Module):
    def __init__(self, args, obs_dim):
        super(MLPBase, self).__init__()
        self._use_feature_normalization = args.use_feature_normalization  # True
        self._use_orthogonal = args.use_orthogonal  # True
        self._use_ReLU = args.use_ReLU  # True
        self.hidden_sizes_list = args.hidden_sizes_list

        if self._use_feature_normalization:
            self.feature_norm = nn.LayerNorm(obs_dim)

        self.mlp = MLPLayer(obs_dim, self.hidden_sizes_list,
                            self._use_orthogonal, self._use_ReLU)
        
    def forward(self, x):
        if self._use_feature_normalization:
            x = self.feature_norm(x)
        x = self.mlp(x)
        return x

In [11]:
class Args:
    def __init__(self):
        self.use_feature_normalization = True
        self.use_orthogonal = True
        self.use_ReLU = True
        self.hidden_sizes_list = [256, 128, 64]

args = Args()
obs_dim = 420

model = MLPBase(args, obs_dim)
print(model)

MLPBase(
  (feature_norm): LayerNorm((420,), eps=1e-05, elementwise_affine=True)
  (mlp): MLPLayer(
    (layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=420, out_features=256, bias=True)
        (1): ReLU()
        (2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      )
      (1): Sequential(
        (0): Linear(in_features=256, out_features=128, bias=True)
        (1): ReLU()
        (2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (2): Sequential(
        (0): Linear(in_features=128, out_features=64, bias=True)
        (1): ReLU()
        (2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
)


In [12]:
def _find_first_5_elements_with_multiple_ones(lst):
    # Loop through the list in reverse order in chunks of 5 elements
    for i in range(len(lst) - 1, 3, -5):
        # Get the current chunk of 5 elements
        chunk = lst[i-4:i+1]
        
        # Check if there is more than one `1` in the current chunk
        if chunk.count(1) > 1:
            return chunk
    
    # If no chunk with more than one `1` is found, return an empty list
    return []

In [13]:
_find_first_5_elements_with_multiple_ones([1,0,0,0,0,  1,0,0,1,1, 0,0,1,0,1])

[0, 0, 1, 0, 1]

In [14]:
import random
chosen_action  = random.choice(range(1, 11))
chosen_action

7

In [15]:
{
    "simpledoorkey":{
        "episode": 100,
        "level": "easy",
        "description": "an agent in a minigrid environment in reinfrocement learning, the task of the agent is toggle the door in the maze with key. please help agent to plan the next action given agent's current observations and statu: carry {object} or none. Availabel actions may includes: explore, go to {object}, pick up {object}, toggle {object}. the actions should be displayed in a list. Do not explain the reasoning. \n ",
        "example": "observation: {observed nothing}, action: {explore}. \n observation: {observed a door}, action: {explore}. \n observation: {observed a key, observed a door}, action: {go to the key, pick up the key, go to the door, toggle the door}. \n observation: {observed a door, carry key}, action: {go to the door, toggle the door}. \n observation: {observed a key}, action: {go to the key, pick up the key, explore}.",
        "configurations": "MiniGrid-SimpleDoorKey-Min5-Max10-View3" 
    },

}

{'simpledoorkey': {'episode': 100,
  'level': 'easy',
  'description': "an agent in a minigrid environment in reinfrocement learning, the task of the agent is toggle the door in the maze with key. please help agent to plan the next action given agent's current observations and statu: carry {object} or none. Availabel actions may includes: explore, go to {object}, pick up {object}, toggle {object}. the actions should be displayed in a list. Do not explain the reasoning. \n ",
  'example': 'observation: {observed nothing}, action: {explore}. \n observation: {observed a door}, action: {explore}. \n observation: {observed a key, observed a door}, action: {go to the key, pick up the key, go to the door, toggle the door}. \n observation: {observed a door, carry key}, action: {go to the door, toggle the door}. \n observation: {observed a key}, action: {go to the key, pick up the key, explore}.',
  'configurations': 'MiniGrid-SimpleDoorKey-Min5-Max10-View3'}}

In [16]:
from abc import ABC, abstractmethod
import requests, os

class Base_Planner(ABC):
    """The base class for Planner."""

    def __init__(self):
        super().__init__()
        self.dialogue_system = ''                  
        self.dialogue_user = ''
        self.dialogue_logger = ''         
        self.show_dialogue = False
        self.llm_model = None
        self.llm_url = None
        
    def reset(self, show=False):
        self.dialogue_user = ''
        self.dialogue_logger = ''
        self.show_dialogue = show
        
    ## initial prompt, write in 'prompt/task_info.json
    def initial_planning(self, decription, example):
        if self.llm_model is None:
            assert "no select Large Language Model"
        prompts = decription + example
        self.dialogue_system += decription + "\n"
        self.dialogue_system += example + "\n"

        ## set system part
        server_error_cnt = 0
        while server_error_cnt<10:
            try:
                url = self.llm_url
                headers = {'Content-Type': 'application/json'}
                
                data = {'model': self.llm_model, "messages":[{"role": "system", "content": prompts}]}
                response = requests.post(url, headers=headers, json=data)
                
                if response.status_code == 200:
                    result = response.json()                    
                    server_flag = 1
                                
                if server_flag:
                    break
                    
            except Exception as e:
                server_error_cnt += 1
                print(e)   
        
        
  

In [17]:
c = Base_Planner()
c.show_dialogue

False

In [18]:
description = ''
example = ''

In [19]:
c.initial_planning(description, example)

Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?
Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?
Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?
Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?
Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?
Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?
Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?
Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?
Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?
Invalid URL 'None': No scheme supplied. Perhaps you meant https://None?
